In [1]:
from typing import List
from abc import ABC, abstractmethod
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()


class TradeRecord(Base):
    __tablename__ = 'TradeRecord'
    id = Column(Integer, primary_key=True)
    source_currency = Column(String)
    dest_currency = Column(String)
    amount = Column(Integer)
    price = Column(Float)

    def __init__(self, source, dest, amount, price):
        self.source_currency = source
        self.dest_currency = dest
        self.amount = amount
        self.price = price
        
    def __str__(self):
        return f'{self.source_currency}, {self.dest_currency}, {self.amount}, {self.price}'
    

In [2]:
class TradeProcessor:
    '''Class for processing trade records'''
    def __init__(self, filename: str) -> None:
        self.filename = filename
        
    def process_trades(self):
        '''read records from the file, process them 
        and store them to a database.'''
        
        #read file
        trade_records: List[str] = []
        with open(self.filename) as data_source:
            for trade_record in data_source: 
                trade_records.append(trade_record)
                
        #parse and validate the individual records
        trades: List[TradeRecord] = []
        for index, line in enumerate(trade_records):
            fields = line.split(',')
            if len(fields) != 3:
                print(f'Line {index} malformed. {len(fields)} field(s) found.')
                continue
            if len(fields[0]) != 6:
                print(f'Trade currencies on line {index} malformed: "{fields[0]}"')
                continue
            try:
                trade_amount = float(fields[1])
            except ValueError:
                print(f"Trade amount on line {index} not a valid integer: '{fields[1]}'")
            
            try:
                trade_price = float(fields[2])
            except ValueError:
                print(f"Trade price on line {index} not a valid decimal:'{fields[2]}'")

            #map validated records to domain objects
            sourceCurrencyCode = fields[0][:3]
            destinationCurrencyCode = fields[0][3:]
            trade = TradeRecord(source=sourceCurrencyCode, dest=destinationCurrencyCode,
                                amount=trade_amount, price=trade_price)
            trades.append(trade)
            
        #store the records to the database
        engine = create_engine('postgresql://postgres:u2402/501@localhost:5432/python')
        Session = sessionmaker(bind=engine)
        Base.metadata.create_all(engine)
        session = Session()
        for trade in trades:
            session.add(trade)
        session.commit()
        session.close()
        print(f'{len(trades)} records have saved')

In [3]:
trade_processor = TradeProcessor('Data.txt')
trade_processor.process_trades()

Trade currencies on line 3 malformed: "UIYUY"
3 records have saved
